In [20]:
# !pip install fastai==2.0
import fastai
# print(fastai.__ version __ ) # should be 2.0.0
from fastai.vision.all import *
import pandas as pd
%matplotlib inline
from fastai.vision.all import *

ModuleNotFoundError: No module named 'fastai.callback.all'; 'fastai.callback' is not a package

In [14]:
import albumentations as Alb
class AlbTransform(Transform):
    def __init__(self, aug): self.aug = aug
    def encodes(self, img: PILImage):
        aug_img = self.aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_augs(): return Alb.Compose([
    Alb.Transpose(),
    Alb.Flip(),
    Alb.HorizontalFlip(),
    Alb.RandomRotate90(),
    Alb.RandomScale(),
    Alb.RandomBrightnessContrast(),
])

item_tfms = [Resize(224), AlbTransform(get_augs())]
batch_tfms = Normalize.from_stats(*imagenet_stats) 

NameError: name 'PILImage' is not defined

In [ ]:
train_path="D:\Comp 6721\Datasets\Dataset3\train"
train_df = pd.read_csv(r"D:\Comp 6721\Datasets\Dataset3\train_df.csv")
train_df['Target'] = train_df['Target'].apply(lambda x: x.strip())
# for interactive DEBUG: reduce amount of train images : train_df = train_df[:512]
dls = ImageDataLoaders.from_df(train_df, path=train_path, suff='-c.png', # label_delim = ' ',
    item_tfms=item_tfms, batch_tfms=batch_tfms, shuffle=True, bs=64, valid_pct=0.)

In [ ]:


# try this to see dataloader working
# dls.train.show_batch(max_n=12)
# dls.valid.show_batch(max_n=12)



In [ ]:


learn = vision_learner(dls, densenet121, path='.', 
    loss_func=FocalLoss(),  
    metrics=[F1ScoreMulti(average='macro'), F1ScoreMulti(average='weighted'), APScoreMulti()] )



In [ ]:


learn.fine_tune(90, freeze_epochs=3) # learn is performed using fit_one_cycle()



In [ ]:
learn.export()

In [ ]:


test_path='../input/xray-body-images-in-png-unifesp-competion/images/test'
test_df = pd.read_csv('../input/xray-body-images-in-png-unifesp-competion/test_df.csv')
tdls = ImageDataLoaders.from_df(test_df, path=test_path, suff='-c.png',
    item_tfms=item_tfms, batch_tfms=batch_tfms, shuffle=False)
tst_dl = tdls.test_dl(test_df)

In [ ]:
preds = learn.tta(dl=tst_dl, n=64, use_max=False)

In [ ]:


# simplest single label submission
# use this when multilabel like '9 21' is considered as a single class
predss = learn.dls.vocab[np.argmax(preds[0], axis=1)]
test_df['Target'] = predss
submission_df = test_df[['SOPInstanceUID', 'Target']]
submission_df.to_csv(f'submission_XRP_argmax.csv', header=True, index=False)
submission_df



In [ ]:
train_df.Target.value_counts()

In [ ]:


test_df.Target.value_counts() 

